In [3]:
import os 
import keras
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
plt.style.use('dark_background')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

One Hot Encoding The Target Classes

In [4]:
encoder = OneHotEncoder()
encoder.fit([[0], [1]])
# 0 = Tumor
# 1 = Normal

OneHotEncoder()